# Dataset-Generator.

In [1]:
import os
import datagenerator
from datagenerator import DataGenerator, get_dataset_path, create_datagenerator_from_parameters
import pickle
import random
import utils
import sys

Using TensorFlow backend.


In [2]:
import multiprocessing
multiprocessing_jobs = multiprocessing.cpu_count()
print("Going to spawn", multiprocessing_jobs, "jobs...")

Going to spawn 8 jobs...


# Setting the parameters.

In [3]:
dataset_path = get_dataset_path()
print("Using dataset path:", dataset_path)

# For creating pointclouds.
dataset_parameters_pointclouds = {}
dataset_parameters_pointclouds["input_type"] = "pointcloud"
dataset_parameters_pointclouds["output_targets"] = ["height", "weight"]    
dataset_parameters_pointclouds["random_seed"] = 666
dataset_parameters_pointclouds["pointcloud_target_size"] = 30000
dataset_parameters_pointclouds["pointcloud_random_rotation"] = True
dataset_parameters_pointclouds["dataset_size_train"] = 1000
dataset_parameters_pointclouds["dataset_size_test"] = 200

# For creating voxelgrids.
dataset_parameters_voxelgrids = {}
dataset_parameters_voxelgrids["input_type"] = "voxelgrid"
dataset_parameters_voxelgrids["output_targets"] = ["height", "weight"]    
dataset_parameters_voxelgrids["random_seed"] = 666
dataset_parameters_voxelgrids["voxelgrid_target_shape"] = (32, 32, 32)
dataset_parameters_voxelgrids["voxel_size_meters"] = 0.1
dataset_parameters_voxelgrids["voxelgrid_random_rotation"] = True
dataset_parameters_voxelgrids["dataset_size_train"] = 6000
dataset_parameters_voxelgrids["dataset_size_test"] = 1000

Using dataset path: /Users/tristanbehrens/Datasets/welthungerhilfe/20180803


In [4]:
dataset_parameters_to_use = []
dataset_parameters_to_use.append(dataset_parameters_pointclouds)
dataset_parameters_to_use.append(dataset_parameters_voxelgrids)

# Analysis.

In [5]:
#analyze = True
analyze = False

In [6]:
datagenerator = create_datagenerator_from_parameters(dataset_path, dataset_parameters_to_use[0])

Creating data-generator...
/Users/tristanbehrens/Datasets/welthungerhilfe/20180803
QR-Code SAM-GOV-025 has 1 different manual measurements
   Target [67, 6.7] with 0 JPGs and 51 PCDs.
QR-Code SAM-SNG-073 has 3 different manual measurements
   Target [82, 8.8] with 0 JPGs and 41 PCDs.
   Target [82, 8.8] with 0 JPGs and 40 PCDs.
   Target [82, 8.5] with 0 JPGs and 0 PCDs.
QR-Code SAM-GOV-014 has 1 different manual measurements
   Target [91.1, 11.1] with 0 JPGs and 77 PCDs.
QR-Code SAM-GOV-087 has 1 different manual measurements
   Target [85.7, 10.2] with 0 JPGs and 60 PCDs.
QR-Code SAM-GOV-004 has 1 different manual measurements
   Target [76.2, 8.3] with 0 JPGs and 71 PCDs.
QR-Code SAM-GOV-026 has 1 different manual measurements
   Target [80.1, 9.2] with 0 JPGs and 47 PCDs.
QR-Code SAM-GOV-068 has 1 different manual measurements
   Target [73.3, 7.4] with 0 JPGs and 39 PCDs.
QR-Code SAM-SNG-013 has 1 different manual measurements
   Target [93.7, 11.2] with 0 JPGs and 0 PCDs.
QR-Cod

In [7]:
if analyze:
    datagenerator.analyze_files()

In [8]:
if analyze:
    datagenerator.analyze_pointclouds()

In [9]:
if analyze:
    datagenerator.analyze_voxelgrids()

# Do the train-test-split and generate.

In [10]:
def split_and_generate(datagenerator, dataset_parameters):

    # Do the split.
    random.seed(dataset_parameters["random_seed"])
    qrcodes_shuffle = datagenerator.qrcodes[:]
    random.shuffle(qrcodes_shuffle)
    split_index = int(0.8 * len(qrcodes_shuffle))
    qrcodes_train = sorted(qrcodes_shuffle[:split_index])
    qrcodes_test = sorted(qrcodes_shuffle[split_index:])
    del qrcodes_shuffle
    print("")

    print("QR-Codes for training:", " ".join(qrcodes_train))
    print("")
    print("QR-Codes for testing:", " ".join(qrcodes_test))
    print("")

    print("Generating training data...")
    dataset_train = next(datagenerator.generate(size=dataset_parameters["dataset_size_train"], qrcodes_to_use=qrcodes_train, yield_file_paths=True, verbose=True, multiprocessing_jobs=multiprocessing_jobs))

    print("Generating testing data...")
    dataset_test = next(datagenerator.generate(size=dataset_parameters["dataset_size_test"], qrcodes_to_use=qrcodes_test, yield_file_paths=True, verbose=True, multiprocessing_jobs=multiprocessing_jobs))

    print("Done.")
    return dataset_train, dataset_test

# Method for saving dataset.

In [11]:
def save_dataset(dataset_train, dataset_test, dataset_parameters):
    print("Saving dataset...")
    datetime_string = utils.get_datetime_string()
    dataset_name = datetime_string + "-" + dataset_parameters["input_type"] + "-dataset.p"
    data = (dataset_train, dataset_test, dataset_parameters)
    print("Size:", sys.getsizeof(data))
    pickle.dump(data, open(dataset_name, "wb"))
    print("Saved " + dataset_name)

# Generate with parameters.

In [12]:
for dataset_parameters in dataset_parameters_to_use:
    print(dataset_parameters["input_type"])
    datagenerator = create_datagenerator_from_parameters(dataset_path, dataset_parameters)
    dataset_train, dataset_test = split_and_generate(datagenerator, dataset_parameters)
    save_dataset(dataset_train, dataset_test, dataset_parameters)    

pointcloud
Creating data-generator...
/Users/tristanbehrens/Datasets/welthungerhilfe/20180803
QR-Code SAM-GOV-025 has 1 different manual measurements
   Target [67, 6.7] with 0 JPGs and 51 PCDs.
QR-Code SAM-SNG-073 has 3 different manual measurements
   Target [82, 8.8] with 0 JPGs and 41 PCDs.
   Target [82, 8.8] with 0 JPGs and 40 PCDs.
   Target [82, 8.5] with 0 JPGs and 0 PCDs.
QR-Code SAM-GOV-014 has 1 different manual measurements
   Target [91.1, 11.1] with 0 JPGs and 77 PCDs.
QR-Code SAM-GOV-087 has 1 different manual measurements
   Target [85.7, 10.2] with 0 JPGs and 60 PCDs.
QR-Code SAM-GOV-004 has 1 different manual measurements
   Target [76.2, 8.3] with 0 JPGs and 71 PCDs.
QR-Code SAM-GOV-026 has 1 different manual measurements
   Target [80.1, 9.2] with 0 JPGs and 47 PCDs.
QR-Code SAM-GOV-068 has 1 different manual measurements
   Target [73.3, 7.4] with 0 JPGs and 39 PCDs.
QR-Code SAM-SNG-013 has 1 different manual measurements
   Target [93.7, 11.2] with 0 JPGs and 0 P

Generating using QR-codes: ['SAM-GOV-001', 'SAM-GOV-004', 'SAM-GOV-005', 'SAM-GOV-008', 'SAM-GOV-011', 'SAM-GOV-013', 'SAM-GOV-014', 'SAM-GOV-023', 'SAM-GOV-025', 'SAM-GOV-026', 'SAM-GOV-033', 'SAM-GOV-034', 'SAM-GOV-035', 'SAM-GOV-037', 'SAM-GOV-038', 'SAM-GOV-042', 'SAM-GOV-043', 'SAM-GOV-044', 'SAM-GOV-045', 'SAM-GOV-046', 'SAM-GOV-049', 'SAM-GOV-050', 'SAM-GOV-051', 'SAM-GOV-052', 'SAM-GOV-054', 'SAM-GOV-063', 'SAM-GOV-064', 'SAM-GOV-066', 'SAM-GOV-067', 'SAM-GOV-068', 'SAM-GOV-069', 'SAM-GOV-073', 'SAM-GOV-089', 'SAM-GOV-090', 'SAM-GOV-099', 'SAM-SNG-011', 'SAM-SNG-013', 'SAM-SNG-014', 'SAM-SNG-015', 'SAM-SNG-016', 'SAM-SNG-018', 'SAM-SNG-059', 'SAM-SNG-061', 'SAM-SNG-066', 'SAM-SNG-067', 'SAM-SNG-068', 'SAM-SNG-070', 'SAM-SNG-075', 'SAM-SNG-076', 'SAM-SNG-081', 'SAM-SNG-083', 'SAM-SNG-085', 'SAM-SNG-086', 'SAM-SNG-088', 'SAM-SNG-091', 'SAM-SNG-095', 'SAM-SNG-096', 'SAM-SNG-097', 'SAM-SNG-098', 'SAM-SNG-099']
Generating using QR-codes: ['SAM-GOV-001', 'SAM-GOV-004', 'SAM-GOV-005',

 63% (79 of 125) |##############         | Elapsed Time: 0:00:58 ETA:   0:00:47

Wrong number of columns at line 26526


 75% (94 of 125) |#################      | Elapsed Time: 0:01:10 ETA:   0:00:21

...

Wrong number of columns at line 26323


 67% (84 of 125) |###############        | Elapsed Time: 0:01:02 ETA:   0:00:30

Wrong number of columns at line 26222


100% (125 of 125) |######################| Elapsed Time: 0:01:32 Time:  0:01:32
100% (125 of 125) |######################| Elapsed Time: 0:01:32 Time:  0:01:32
100% (125 of 125) |######################| Elapsed Time: 0:01:33 Time:  0:01:33
100% (125 of 125) |######################| Elapsed Time: 0:01:33 Time:  0:01:33
100% (125 of 125) |######################| Elapsed Time: 0:01:33 Time:  0:01:33
100% (125 of 125) |######################| Elapsed Time: 0:01:33 Time:  0:01:33
100% (125 of 125) |######################| Elapsed Time: 0:01:33 Time:  0:01:33
100% (125 of 125) |######################| Elapsed Time: 0:01:33 Time:  0:01:33


Done.
Saving dataset...
Size: 72
Saved 20180810-1330-voxelgrid-dataset.p
